In [ ]:
!pip install pyLDAvis

In [ ]:
import pandas as pd
import nltk
import re, string, random
import warnings

from collections import Counter
from spacy.matcher import Matcher 
from numpy import nan
from nltk.sentiment import SentimentIntensityAnalyzer
from statistics import mean
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
from pprint import pprint

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


nlp = spacy.load("en_core_web_sm")

nltk.download([
     "names",
     'wordnet',
     "stopwords",
     "state_union",
     "twitter_samples",
     "movie_reviews",
     "averaged_perceptron_tagger",
     "vader_lexicon",
     "punkt",
     'omw-1.4',
     'universal_tagset', ])

warnings.filterwarnings('ignore',category=DeprecationWarning)
# nlp.max_length = 1850000

In [3]:
df = pd.read_csv("/content/Leverage_Collaboration_data_analysis.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ResponseId  304 non-null    object
 1   Q19         214 non-null    object
 2   Q51         204 non-null    object
 3   Q52         201 non-null    object
 4   Q57         188 non-null    object
 5   Q55         184 non-null    object
 6   Q58         184 non-null    object
 7   Q59         183 non-null    object
dtypes: object(8)
memory usage: 19.1+ KB


In [4]:

def word_frequency(doc): 
  words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct] 
  word_freq = Counter(words)
  print("-----COMMON WORDS FREQUENCY ------") 
  pprint(word_freq.most_common(30))

def match_phrase_pattern(doc):
  matcher = Matcher(nlp.vocab) 
  pattern = [{'POS':'NOUN'}, {'POS':'NOUN'}]
  pattern1 = [{'POS':'NOUN'}, {'POS':'VERB'}]
  pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
  pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

  matcher.add('PATTERN', [pattern]) 
  matcher.add('PATTERN1', [pattern1])
  matcher.add('RULE4',[pattern4])
  matcher.add('RULE5',[pattern5]) 
  matches = matcher(doc, as_spans=True) 
  phrases = []
  unique_phrases = [] 
  for span in matches:
      if span.text.lower() not in phrases:
        unique_phrases.append(span.text.lower())
      phrases.append(span.text.lower()) 

  phrase_freq = Counter(phrases)
  print("-----COMMON PHRASE FREQUENCY ------")
  pprint(phrase_freq.most_common(20))

def run_analysis(resp):
  all_text = resp.str.cat(sep = ' ') 
  doc = nlp(all_text)
  word_frequency(doc)
  match_phrase_pattern(doc)  

## **Word Count and Phrase Counts**

In [5]:
# QUESTION 51 ANALYSIS
# What kinds of information would be useful to you for evaluating an 
# individual participant's contributions to a collaborative group?
run_analysis(df.Q51)

-----COMMON WORDS FREQUENCY ------
[('group', 81),
 ('work', 75),
 ('time', 43),
 ('contribute', 43),
 ('contribution', 42),
 (' ', 40),
 ('individual', 40),
 ('project', 31),
 ('know', 27),
 ('member', 26),
 ('effort', 25),
 ('spend', 25),
 ('idea', 25),
 ('person', 23),
 ('useful', 23),
 ('information', 20),
 ('participant', 20),
 ('meeting', 19),
 ('participate', 17),
 ('complete', 16),
 ('evaluate', 16),
 ('think', 16),
 ('collaboration', 15),
 ('team', 14),
 ('peer', 14),
 ('try', 13),
 ('task', 13),
 ('discussion', 12),
 ('people', 12),
 ('help', 12)]
-----COMMON PHRASE FREQUENCY ------
[('work done', 16),
 ('group members', 13),
 ('time spent', 12),
 ('group finish', 6),
 ('group meetings', 6),
 ('peer reviews', 6),
 ('group meeting', 4),
 ('work produced', 4),
 ('lines written', 4),
 ('person contribute', 4),
 ('person did', 4),
 ('group members can participate', 3),
 ('members can participate', 3),
 ('issues came', 2),
 ('time spend', 2),
 ('work portioned', 2),
 ('participant

In [6]:
# QUESTION 52 ANALYSIS
# What kinds of information would be useful to you for evaluating collaboration quality at a group level?
run_analysis(df.Q52)

-----COMMON WORDS FREQUENCY ------
[('group', 135),
 ('work', 67),
 ('collaboration', 60),
 ('member', 52),
 (' ', 44),
 ('quality', 43),
 ('project', 26),
 ('time', 25),
 ('level', 24),
 ('people', 21),
 ('good', 21),
 ('idea', 21),
 ('useful', 21),
 ('team', 20),
 ('product', 19),
 ('individual', 19),
 ('contribute', 17),
 ('final', 15),
 ('information', 15),
 ('evaluate', 15),
 ('progress', 15),
 ('task', 14),
 ('complete', 13),
 ('overall', 13),
 ('person', 13),
 ('like', 12),
 ('think', 12),
 ('spend', 12),
 ('know', 10),
 ('goal', 10)]
-----COMMON PHRASE FREQUENCY ------
[('collaboration quality', 19),
 ('group members', 14),
 ('group level', 11),
 ('time spent', 8),
 ('group works', 6),
 ('work completed', 6),
 ('work done', 6),
 ('group member', 6),
 ('group dynamic', 4),
 ('group stayed', 4),
 ('people get', 4),
 ('person contributed', 4),
 ('progress made', 4),
 ('end product', 4),
 ('team member', 3),
 ('product look', 2),
 ('projects stitched', 2),
 ('team had', 2),
 ('grou

In [7]:
# QUESTION 57 ANALYSIS
# What kind of collaboration-quality information would you like to see in real-time?
run_analysis(df.Q57)

-----COMMON WORDS FREQUENCY ------
[('group', 53),
 ('work', 50),
 ('collaboration', 43),
 ('time', 39),
 ('like', 37),
 (' ', 36),
 ('member', 32),
 ('people', 28),
 ('real', 24),
 ('quality', 19),
 ('information', 19),
 ('progress', 18),
 ('person', 17),
 ('task', 16),
 ('participate', 13),
 ('team', 13),
 ('talk', 13),
 ('discussion', 12),
 ('help', 12),
 ('contribute', 11),
 ('level', 11),
 ('idea', 11),
 ('sure', 11),
 ('project', 10),
 ('speak', 10),
 ('different', 10),
 ('job', 10),
 ('think', 10),
 ('problem', 9),
 ('collaborate', 9)]
-----COMMON PHRASE FREQUENCY ------
[('group members', 10),
 ('quality information', 8),
 ('group finish', 6),
 ('team members', 5),
 ('collaboration quality', 5),
 ('time collaboration', 4),
 ('group member', 4),
 ('work done', 4),
 ('time spent', 4),
 ('peer reviews', 4),
 ('problem set', 4),
 ('time include', 4),
 ('group members can participate', 3),
 ('members can participate', 3),
 ('task progress', 2),
 ('group understand', 2),
 ('informati

In [8]:
# QUESTION 55 ANALYSIS
# How would you use this real-time information?
run_analysis(df.Q55)

-----COMMON WORDS FREQUENCY ------
[('group', 65),
 ('use', 56),
 ('collaboration', 52),
 ('information', 43),
 ('work', 42),
 ('time', 36),
 (' ', 32),
 ('member', 31),
 ('people', 26),
 ('real', 25),
 ('help', 19),
 ('participate', 17),
 ('know', 16),
 ('need', 15),
 ('adjust', 12),
 ('team', 12),
 ('task', 12),
 ('try', 12),
 ('improve', 12),
 ('progress', 12),
 ('project', 11),
 ('understand', 11),
 ('talk', 11),
 ('encourage', 11),
 ('good', 10),
 ('collaborate', 10),
 ('job', 10),
 ('quality', 10),
 ('evaluate', 10),
 ('idea', 9)]
-----COMMON PHRASE FREQUENCY ------
[('time information', 19),
 ('group members', 12),
 ('group finish', 6),
 ('people engaged', 4),
 ('people have', 4),
 ('collaboration quality', 4),
 ('points made', 4),
 ('group members can participate', 3),
 ('members can participate', 3),
 ('tasks tie', 2),
 ('individuals talk', 2),
 ('deadlines based', 2),
 ('group dynamic', 2),
 ('people disengaged', 2),
 ('communication makes', 2),
 ('project went', 2),
 ('other

In [9]:
# QUESTION 58 ANALYSIS
# What kind of collaboration-quality information would you like to see after the collaboration?
run_analysis(df.Q58)

-----COMMON WORDS FREQUENCY ------
[('collaboration', 66),
 ('group', 53),
 ('like', 45),
 ('member', 43),
 ('work', 37),
 (' ', 33),
 ('project', 21),
 ('feel', 20),
 ('information', 19),
 ('people', 17),
 ('think', 17),
 ('time', 16),
 ('quality', 16),
 ('individual', 14),
 ('contribute', 14),
 ('result', 13),
 ('person', 13),
 ('team', 11),
 ('feedback', 11),
 ('go', 11),
 ('overall', 10),
 ('final', 10),
 ('want', 10),
 ('improve', 9),
 ('good', 9),
 ('peer', 9),
 ('maybe', 9),
 ('job', 9),
 ('product', 9),
 ('contribution', 9)]
-----COMMON PHRASE FREQUENCY ------
[('group members', 13),
 ('collaboration went', 8),
 ('group finish', 6),
 ('work done', 6),
 ('team members', 4),
 ('member thought', 4),
 ('people felt', 4),
 ('member contributed', 4),
 ('peer reviews', 4),
 ('quality information', 3),
 ('group members can participate', 3),
 ('members can participate', 3),
 ('time taken', 2),
 ('team felt', 2),
 ('member felt', 2),
 ('collaboration quality', 2),
 ('work contributed', 2

In [10]:
# QUESTION 59 ANALYSIS
# How would you use this post-hoc information?
run_analysis(df.Q59)

-----COMMON WORDS FREQUENCY ------
[('group', 65),
 ('collaboration', 64),
 ('use', 58),
 ('work', 44),
 ('information', 33),
 ('improve', 32),
 (' ', 32),
 ('future', 30),
 ('time', 26),
 ('people', 22),
 ('member', 20),
 ('well', 20),
 ('project', 18),
 ('try', 18),
 ('help', 17),
 ('post', 16),
 ('need', 15),
 ('good', 15),
 ('hoc', 14),
 ('know', 13),
 ('go', 13),
 ('think', 12),
 ('team', 12),
 ('evaluate', 12),
 ('way', 11),
 ('collaborate', 11),
 ('individual', 11),
 ('understand', 10),
 ('learn', 10),
 ('job', 9)]
-----COMMON PHRASE FREQUENCY ------
[('group members', 7),
 ('group finish', 6),
 ('group work', 5),
 ('group member', 4),
 ('group dynamics', 4),
 ('group members can participate', 3),
 ('members can participate', 3),
 ('collaboration quality', 3),
 ('group using', 2),
 ('person thought', 2),
 ('work moves', 2),
 ('team members', 2),
 ('team improve', 2),
 ('group knows', 2),
 ('end product', 2),
 ('time contributing', 2),
 ('evaluation indicates', 2),
 ('individual 

In [ ]:
# pattern2 = [{'POS':'VERB'}, {"POS": "DET", "OP" : "*"},{'POS':'NOUN'}]
# pattern3 = [{'POS':'VERB'}, {"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'NOUN'}]  
# matcher.add('VRB_PHRASE', [pattern2]) 
# matcher.add('VRB_PHRASE', [pattern3]) 

#pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
#pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]
#matcher.add('RULE4',[pattern4])
#matcher.add('RULE5',[pattern5]) 

## **Sentiment Analysis**

In [ ]:
# ---------------------------------SENTIMENT ANALYSIS------------------------------
# USING TWITTER SAMPLES TO AS TRAINIG DATA 

In [11]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    #print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    #print("Accuracy is:", classify.accuracy(classifier, test_data))

    #print(classifier.show_most_informative_features(10))

In [12]:
#SENTIMENT ANALYSIS FOR Q51,Q55,Q57.Q58.Q59
res = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
print("sno","      Q51","       Q52","        Q55","        Q57","        Q58","        Q59      ","pos:neg")
for i in range(len(df.Q51)):
  positives = 0
  negatives = 0
  if df.Q51[i] is not nan and df.Q52[i] is not nan and df.Q55[i] is not nan and df.Q57[i] is not nan and df.Q58[i] is not nan and df.Q59[i] is not nan:
      custom_tokens = remove_noise(word_tokenize(df.Q51[i]))
      Q51 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q51 == 'Positive':
        res[0][0]+=1
        positives+=1
      else:
        res[0][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q52[i]))
      Q52 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q52 == 'Positive':
        res[1][0]+=1
        positives+=1
      else:
        res[1][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q55[i]))
      Q55 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q55 == 'Positive':
        res[2][0]+=1
        positives+=1
      else:
        res[2][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q57[i]))
      Q57 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q57 == 'Positive':
        res[3][0]+=1
        positives+=1
      else:
        res[3][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q58[i]))
      Q58 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q58 == 'Positive':
        res[4][0]+=1
        positives+=1
      else:
        res[4][1]+=1
        negatives+=1
      

      custom_tokens = remove_noise(word_tokenize(df.Q59[i]))
      Q59 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q59 == 'Positive':
        res[5][0]+=1
        positives+=1
      else:
        res[5][1]+=1
        negatives+=1
    
      print(i+1,"  ",Q51,"  ",Q52,"  ",Q55,"  ",Q57,"  ",Q58,"  ",Q59,"    ",positives,":",negatives)

sno       Q51        Q52         Q55         Q57         Q58         Q59       pos:neg
11    Negative    Positive    Negative    Positive    Positive    Positive      4 : 2
12    Negative    Positive    Positive    Positive    Positive    Positive      5 : 1
13    Positive    Positive    Positive    Positive    Positive    Positive      6 : 0
14    Negative    Positive    Negative    Positive    Negative    Negative      2 : 4
15    Positive    Positive    Positive    Positive    Negative    Positive      5 : 1
16    Positive    Positive    Negative    Positive    Positive    Negative      4 : 2
17    Negative    Negative    Negative    Negative    Negative    Negative      0 : 6
18    Positive    Positive    Negative    Positive    Positive    Positive      5 : 1
19    Positive    Positive    Positive    Negative    Positive    Positive      5 : 1
20    Positive    Positive    Negative    Positive    Positive    Positive      5 : 1
22    Negative    Negative    Negative    Positive   

In [13]:
print("pos/neg     ","Q51    ","Q52     ","Q55     ","Q57     ","Q58       ","Q59")
print("Positives   ",res[0][0],"    ",res[1][0],"    ",res[2][0],"    ",res[3][0],"    ",res[4][0],"      ",res[5][0])
print("Negatives   ",res[0][1],"    ",res[1][1],"     ",res[2][1],"     ",res[3][1],"     ",res[4][1],"       ",res[5][1])

pos/neg      Q51     Q52      Q55      Q57      Q58        Q59
Positives    134      128      109      117      119        128
Negatives    46      52       71       63       61         52


In [14]:
#SENTIMENT ANALYSIS FOR Q19, Q51,Q55,Q57.Q58.Q59
res = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
print("sno","     Q19","          Q51","         Q52","      Q55","          Q57","         Q58","        Q59","       pos:neg")
for i in range(len(df.Q51)): 
  positives = 0
  negatives = 0
  if df.Q19[i] is not nan and df.Q51[i] is not nan and df.Q52[i] is not nan and df.Q55[i] is not nan and df.Q57[i] is not nan and df.Q58[i] is not nan and df.Q59[i] is not nan:
      custom_tokens = remove_noise(word_tokenize(df.Q19[i]))
      Q19 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q19 == 'Positive':
        res[0][0]+=1
        positives+=1
      else:
        res[0][1]+=1
        negatives+=1

      custom_tokens = remove_noise(word_tokenize(df.Q51[i]))
      Q51 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q51 == 'Positive':
        res[1][0]+=1
        positives+=1
      else:
        res[1][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q52[i]))
      Q52 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q52 == 'Positive':
        res[2][0]+=1
        positives+=1
      else:
        res[2][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q55[i]))
      Q55 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q55 == 'Positive':
        res[3][0]+=1
        positives+=1
      else:
        res[3][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q57[i]))
      Q57 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q57 == 'Positive':
        res[4][0]+=1
        positives+=1
      else:
        res[4][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q58[i]))
      Q58 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q58 == 'Positive':
        res[5][0]+=1
        positives+=1
      else:
        res[5][1]+=1
        negatives+=1
      

      custom_tokens = remove_noise(word_tokenize(df.Q59[i]))
      Q59 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q59 == 'Positive':
        res[6][0]+=1
        positives+=1
      else:
        res[6][1]+=1
        negatives+=1
    
      print(i+1,"  ",Q19,"   ",Q51,"   ",Q52,"   ",Q55,"   ",Q57,"   ",Q58,"   ",Q59,"   ",positives,":",negatives)

sno      Q19           Q51          Q52       Q55           Q57          Q58         Q59        pos:neg
11    Positive     Negative     Positive     Negative     Positive     Positive     Positive     5 : 2
12    Positive     Negative     Positive     Positive     Positive     Positive     Positive     6 : 1
13    Positive     Positive     Positive     Positive     Positive     Positive     Positive     7 : 0
14    Negative     Negative     Positive     Negative     Positive     Negative     Negative     2 : 5
15    Negative     Positive     Positive     Positive     Positive     Negative     Positive     5 : 2
16    Positive     Positive     Positive     Negative     Positive     Positive     Negative     5 : 2
17    Positive     Negative     Negative     Negative     Negative     Negative     Negative     1 : 6
18    Negative     Positive     Positive     Negative     Positive     Positive     Positive     5 : 2
19    Negative     Positive     Positive     Positive     Negative     P

In [15]:
print("pos/neg    ","    Q19  "," Q51  ","  Q52","    Q55  ","  Q57  ","  Q58  ","  Q59   ")
print("Positives    "," ",res[0][0],"   ",res[1][0],"   ",res[2][0],"   ",res[3][0],"   ",res[4][0],"   ",res[5][0],"   ",res[6][0],"    ")
print("Negatives    "," ",res[0][0],"   ",res[1][1],"    ",res[2][1],"    ",res[3][1],"    ",res[4][1],"    ",res[5][1],"    ",res[6][1],"    ")

pos/neg         Q19    Q51     Q52     Q55     Q57     Q58     Q59   
Positives       81     128     122     103     109     114     121     
Negatives       81     43      49      68      62      57      50     


## **Topic Modeling**

In [16]:
stopwords = nltk.corpus.stopwords.words("english")
matcher = Matcher(nlp.vocab) 
pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

matcher.add('RULE4',[pattern4])
matcher.add('RULE5',[pattern5]) 

In [17]:

#Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    #stop_words.extend(['much','find','past','see','want','way','also','else','get','put','often','kind','one'])
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  
def make_bigrams(data_words,data_words_nostops):
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in data_words_nostops]

def make_trigrams(data_words,data_words_nostops):
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram[data_words], threshold=5) 
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in data_words_nostops]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
      #print(sent)
      doc = nlp(" ".join(sent))
      token_lemma = [token.lemma_ for token in doc]
      doc_lemma = nlp(" ".join(token_lemma))
      matches = matcher(doc_lemma, as_spans=True)
      if(len(matches) != 0):   
        #print(matches,'matches')
        unique_phrases = [] 
        for span in matches:
            if span.text.lower() not in unique_phrases:
              unique_phrases.append(span.text.lower().replace(" ","_")) 
        texts_out.append(unique_phrases)
        #texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def lemmatization_old(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
      doc = nlp(" ".join(sent))
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
def run_topic_modeling(all_text,mode='phrase'):
 
  data = nltk.sent_tokenize(all_text)

  # Remove Emails
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

  # Remove new line characters
  data = [re.sub('\s+', ' ', sent) for sent in data]

  # Remove distracting single quotes
  data = [re.sub("\'", "", sent) for sent in data]


  data_words = list(sent_to_words(data))

  # See trigram example
  #print(trigram_mod[bigram_mod[data_words[100]]])

  # Remove Stop Words
  data_words_nostops = remove_stopwords(data_words)

  # Form Trigrams
  #data_words_bigrams = make_bigrams(data_words_nostops)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  #nlp = spacy.load('en', disable=['parser', 'ner'])

  id2word = None
  lda_model = None
  corpus = None

  if mode == 'phrase':
    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)
    # Create Corpus
    texts = data_lemmatized
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
  else:
      # Form Bigrams
      data_words_bigrams = make_bigrams(data_words,data_words_nostops)
      data_lemmatized = lemmatization_old(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
      # Create Dictionary
      id2word = corpora.Dictionary(data_lemmatized)
      # Create Corpus
      texts = data_words_bigrams
      # Term Document Frequency
      corpus = [id2word.doc2bow(text) for text in texts]


  # Build LDA model
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20, 
                                            random_state=100,
                                            update_every=1,
                                            #chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)


  return lda_model,corpus,id2word
  

In [19]:
#------topic modeling for question 51 -----------
lda_model,corpus,id2word = run_topic_modeling(df.Q51.str.cat(sep = ' '),'text')
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.040432  0.240106       1        1  10.123292
12     0.010165 -0.248053       2        1   9.111039
13    -0.195326 -0.158727       3        1   8.872130
18    -0.047212  0.246996       4        1   7.438205
17     0.139183 -0.204253       5        1   6.842238
1      0.306452 -0.111020       6        1   6.507816
11     0.156304  0.202085       7        1   5.608133
16    -0.031952 -0.148299       8        1   4.703645
5     -0.189327  0.023573       9        1   4.659345
19     0.089896  0.092926      10        1   4.543063
3      0.195196  0.051880      11        1   4.515343
7     -0.007451  0.097034      12        1   4.269839
15    -0.264411 -0.017702      13        1   3.821724
8     -0.104607  0.095575      14        1   3.496416
10     0.062122 -0.087945      15        1   3.345286
4     -0.164605  0.178477      16        1   3.217775
14    -0.095096 -0.039450      17        1   2.615847
2     -0.101918 -0.162152      18        1   2.336670
9      0.048883 -0.012133      19        1   2.092118
6      0.153271 -0.038919      20        1   1.880077, topic_info=           Term       Freq      Total Category  logprob  loglift
69       useful  16.000000  16.000000  Default  30.0000  30.0000
20        group  43.000000  43.000000  Default  29.0000  29.0000
27         work  39.000000  39.000000  Default  28.0000  28.0000
78          job   7.000000   7.000000  Default  27.0000  27.0000
79  participate   9.000000   9.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
3      actually   0.026174   4.624065  Topic20  -6.9083  -1.2004
4       already   0.026174   1.494095  Topic20  -6.9083  -0.0706
5       certain   0.026174   2.211236  Topic20  -6.9083  -0.4627
6        choice   0.026174   1.219076  Topic20  -6.9083   0.1328
7       explain   0.026174   2.204850  Topic20  -6.9083  -0.4598

[911 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
488      18  0.804846  ability
148       2  0.224902     able
148       3  0.449804     able
148      12  0.224902     able
148      14  0.224902     able
...     ...       ...      ...
344       3  0.581116  working
344       9  0.193705  working
344      18  0.193705  working
533       2  0.669356    worse
340       2  0.669572    write

[934 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 13, 14, 19, 18, 2, 12, 17, 6, 20, 4, 8, 16, 9, 11, 5, 15, 3, 10, 7])

# **Concordance**

In [33]:
all_text = df.Q51.str.cat(sep = ' ') 
text = nltk.Text(nltk.word_tokenize(all_text))
text.concordance("collaboration")

Displaying 15 of 15 matches:
ow what their goals were for the collaboration and what steps they took as an i
what . How often did they attend collaboration sessions . How communicative are
given assignment or project in a collaboration . Information such as communicat
ead to more competition and less collaboration . How much time was put into the
laborative effort outside of the collaboration time . Knowing how the others in
and type of things said during a collaboration session . Another useful measure
 prepared they were entering the collaboration . I think it would be useful to 
 they exactly contributed to the collaboration and how they contributed it . Pe
r group members felt about their collaboration I do n't think quantitative meas
goal . the different segments of collaboration How often they speak and their c
heir contributions accordingly ( collaboration is n't just measured by how many
tion ) How others perceive their collaboration , how they felt about their own 
 , how they

In [ ]:
#------topic modeling for question 52 -----------
lda_model,corpus,id2word = run_topic_modeling(df.Q52.str.cat(sep = ' '))
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.256005 -0.075259       1        1  9.425267
3      0.217943  0.148278       2        1  9.187938
1     -0.229162  0.134358       3        1  7.995796
9     -0.240797 -0.050152       4        1  7.335365
0      0.112080 -0.210976       5        1  7.069125
11    -0.059256 -0.227008       6        1  6.943204
7     -0.156188 -0.154606       7        1  5.695559
6      0.030757  0.214709       8        1  5.527992
16     0.093992  0.123177       9        1  4.337397
14     0.022054 -0.142681      10        1  4.094372
15     0.122886 -0.074204      11        1  3.896541
18    -0.089261  0.133180      12        1  3.799786
8      0.135167  0.030058      13        1  3.775288
13    -0.115081  0.050818      14        1  3.601657
2     -0.023586  0.150295      15        1  3.601656
5     -0.006894  0.052403      16        1  3.207995
17    -0.057554 -0.064704      17        1  3.036163
19    -0.071897 -0.010545      18        1  2.874449
12     0.056544 -0.017742      19        1  2.587491
4      0.002246 -0.009400      20        1  2.006960, topic_info=                    Term      Freq     Total Category  logprob  loglift
30            time_spend  4.000000  4.000000  Default  30.0000  30.0000
73           group_spend  2.000000  2.000000  Default  29.0000  29.0000
10             idea_come  1.000000  1.000000  Default  28.0000  28.0000
66     member_contribute  1.000000  1.000000  Default  27.0000  27.0000
77     person_contribute  2.000000  2.000000  Default  26.0000  26.0000
..                   ...       ...       ...      ...      ...      ...
23  result_product_weigh  0.016792  1.146009  Topic20  -5.8551  -0.3146
24    status_communicate  0.016792  1.146009  Topic20  -5.8551  -0.3146
25        group_consider  0.016792  0.948683  Topic20  -5.8551  -0.1256
26   work_group_consider  0.016792  0.948683  Topic20  -5.8551  -0.1256
27     group_member_come  0.016792  0.963164  Topic20  -5.8551  -0.1408

[914 rows x 6 columns], token_table=      Topic      Freq                        Term
term                                             
231       7  0.973930         ability_communicate
31        9  1.004560                 ability_get
69        7  0.973930               act_idea_find
188       3  0.891864         admit_mistake_comes
106       7  0.973930  allocate_person_contribute
...     ...       ...                         ...
151       1  0.562466                   work_make
151       2  0.562466                   work_make
64        7  0.973930                 work_oppose
36        2  0.885201                    work_put
162       5  0.911583               work_time_use

[190 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 4, 2, 10, 1, 12, 8, 7, 17, 15, 16, 19, 9, 14, 3, 6, 18, 20, 13, 5])

In [34]:
#------topic modeling for question 57 -----------
lda_model,corpus,id2word = run_topic_modeling(df.Q57.str.cat(sep = ' '),'text')
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18     0.210672 -0.089458       1        1  8.880099
7      0.194418  0.124499       2        1  8.255668
4     -0.248666 -0.008394       3        1  8.101527
13    -0.185810 -0.081660       4        1  7.717222
9     -0.058358 -0.288206       5        1  7.518483
12     0.078379 -0.173983       6        1  6.393221
6     -0.108119 -0.169380       7        1  6.270998
1      0.220868  0.019801       8        1  5.918350
16     0.071979  0.200663       9        1  5.648234
8      0.137440 -0.160279      10        1  5.602604
19    -0.049792 -0.111879      11        1  5.013132
10     0.106866  0.111837      12        1  4.719415
3      0.106536 -0.010259      13        1  4.294821
0     -0.148847  0.177146      14        1  4.041076
15    -0.042514  0.231608      15        1  3.448516
2     -0.096858  0.064008      16        1  2.579582
17    -0.002735 -0.033551      17        1  2.308769
14    -0.139795  0.094103      18        1  2.142427
5     -0.018986  0.056852      19        1  0.788893
11    -0.026679  0.046533      20        1  0.356961, topic_info=              Term       Freq      Total Category  logprob  loglift
65             job   7.000000   7.000000  Default  30.0000  30.0000
6         everyone  11.000000  11.000000  Default  29.0000  29.0000
126         person  12.000000  12.000000  Default  28.0000  28.0000
44   collaboration  22.000000  22.000000  Default  27.0000  27.0000
13        progress  15.000000  15.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
24      discussion   0.007617   8.537485  Topic20  -6.2442  -1.3865
25          moment   0.007617   2.053198  Topic20  -6.2442   0.0386
26           speak   0.007617   2.033869  Topic20  -6.2442   0.0480
27    conversation   0.007617   4.761193  Topic20  -6.2442  -0.8025
29       direction   0.007617   1.350014  Topic20  -6.2442   0.4579

[927 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
219       3  0.569201          able
219       6  0.284601          able
219      11  0.284601          able
426      16  0.864551    accomplish
250      14  0.784128  accomplished
...     ...       ...           ...
58       13  0.094721       working
58       17  0.094721       working
355       2  0.498106       writing
355      15  0.498106       writing
143       4  0.727308           yet

[778 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 8, 5, 14, 10, 13, 7, 2, 17, 9, 20, 11, 4, 1, 16, 3, 18, 15, 6, 12])

# **Concordance**

In [36]:
all_text_52 = df.Q52.str.cat(sep = ' ') 
text = nltk.Text(nltk.word_tokenize(all_text_52))
text.concordance("progress")

Displaying 15 of 15 matches:
be the final idea . Seeing the the progress and work completed after a meeting
ng document that clearly shows the progress done on the collaboration would be
ess A time table that measures the progress of people 's work during the proje
formation learned or the amount of progress made within the task at hand . If 
hand . If there is n't significant progress , there should be an understanding
dual needs to work on to make that progress . Individual contributions , group
y be helpful I would check out the progress made on the main task . I think pe
re present and up to date with the progress . Feedback from within the group i
accomplish or at least making good progress towards that . Efforts by the team
ceed with can be a good measure of progress . Also , seeing whether the group 
seeing whether the group is making progress on the overall project can be a go
ment of every group member and the progress of the group project or work can h
s given to each person 

In [ ]:
#------topic modeling for question 55 -----------
lda_model,corpus,id2word = run_topic_modeling(df.Q55.str.cat(sep = ' '))
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.118483  0.237209       1        1  9.891084
17     0.070491 -0.258894       2        1  9.154142
8      0.154121  0.194166       3        1  7.562211
3     -0.106294 -0.211866       4        1  6.980277
7     -0.205848  0.100813       5        1  6.651379
10     0.031129  0.219954       6        1  6.521493
4      0.225190 -0.036910       7        1  6.015780
19    -0.212856 -0.046023       8        1  5.806319
11     0.172337  0.085333       9        1  5.075266
16    -0.068203  0.106381      10        1  4.824856
6     -0.151769 -0.108547      11        1  4.533573
14     0.021902 -0.143025      12        1  4.286866
15    -0.125626  0.013924      13        1  4.239435
2      0.117841 -0.124416      14        1  4.179536
9      0.123843 -0.042069      15        1  3.574243
18     0.067198  0.011276      16        1  2.853144
1      0.026096  0.063067      17        1  2.853143
13    -0.021391 -0.052564      18        1  2.511165
5     -0.005175 -0.006622      19        1  1.394412
12     0.005497 -0.001188      20        1  1.091676, topic_info=                           Term      Freq     Total Category  logprob  loglift
18                      job_try  2.000000  2.000000  Default  30.0000  30.0000
19          participate_job_try  2.000000  2.000000  Default  29.0000  29.0000
21               finish_job_try  1.000000  1.000000  Default  28.0000  28.0000
31                people_engage  1.000000  1.000000  Default  27.0000  27.0000
59        information_would_use  1.000000  1.000000  Default  26.0000  26.0000
..                          ...       ...       ...      ...      ...      ...
23    aspect_collaboration_feel  0.010841  1.051695  Topic20  -5.6595  -0.0574
24           collaboration_feel  0.010841  1.051695  Topic20  -5.6595  -0.0574
25  progress_project_would_good  0.010841  1.080650  Topic20  -5.6595  -0.0846
26           project_would_good  0.010841  1.080650  Topic20  -5.6595  -0.0846
28                meeting_think  0.010841  1.027871  Topic20  -5.6595  -0.0345

[915 rows x 6 columns], token_table=      Topic      Freq                     Term
term                                          
183       5  0.947692  action_information_give
176       3  0.925640        address_help_know
148      13  1.037981             adjust_think
238       3  0.925369    amount_work_do_adjust
133       2  0.882331         array_contribute
...     ...       ...                      ...
253      12  1.032600                 work_see
45        2  0.882331         work_try_improve
91        6  0.650215           work_would_use
91        9  0.650215           work_would_use
93       13  1.037981          workload_ensure

[231 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 18, 9, 4, 8, 11, 5, 20, 12, 17, 7, 15, 16, 3, 10, 19, 2, 14, 6, 13])

# **Concordance**

In [37]:
all_text_55 = df.Q55.str.cat(sep = ' ') 
text = nltk.Text(nltk.word_tokenize(all_text_55))
text.concordance("progress")

Displaying 12 of 12 matches:
lestones . Being able to track the progress of a project would be good because
 use this information to gauge the progress and status of this collaboration t
 should be warned or removed . The progress updates would just be for my peace
n how related a given project is . Progress allows someone to understand how f
working on the question or to make progress with the new information at hand .
 the group moves forward and makes progress . To adjust behaviors to improve c
em a bit more useful By seeing the progress we make on the main task , I would
d and compare this with the actual progress of the team improve the group 's c
oup 's work style . We can use the progress checks to figure out of the member
eds to be changed I will adjust my progress based on his progress and I will s
ll adjust my progress based on his progress and I will see whether I can solve
ormation would be used to evaluate progress and communicate with collaborators


In [38]:
#------topic modeling for question 57 -----------
lda_model,corpus,id2word = run_topic_modeling(df.Q57.str.cat(sep = ' '))
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.225348 -0.123715       1        1  9.334786
9     -0.243811  0.042071       2        1  8.305906
18     0.081700 -0.240076       3        1  7.470220
12    -0.078542 -0.247042       4        1  6.703978
8      0.110979  0.223422       5        1  6.682776
15    -0.077615  0.228163       6        1  6.546828
14    -0.166967  0.129735       7        1  5.766897
19    -0.092606 -0.121276       8        1  5.058500
13     0.089956 -0.136462       9        1  4.736355
4      0.168249 -0.093227      10        1  4.523320
2      0.201639 -0.003275      11        1  4.485657
17     0.181316  0.087006      12        1  4.362326
1     -0.112572 -0.022208      13        1  3.936131
16    -0.061756  0.063396      14        1  3.767078
10    -0.015538  0.132488      15        1  3.737154
7      0.088704  0.124666      16        1  3.628375
5      0.023469 -0.073721      17        1  3.270609
11     0.099282  0.010307      18        1  3.163416
0      0.032925  0.026076      19        1  2.707477
6     -0.003466 -0.006329      20        1  1.812210, topic_info=                                  Term      Freq     Total Category  logprob  \
9                              job_try  3.000000  3.000000  Default  30.0000   
73                          time_spend  2.000000  2.000000  Default  29.0000   
10                 participate_job_try  2.000000  2.000000  Default  28.0000   
74                         people_work  1.000000  1.000000  Default  27.0000   
201                        people_talk  1.000000  1.000000  Default  26.0000   
..                                 ...       ...       ...      ...      ...   
19   organization_communication_assign  0.015507  1.036418  Topic20  -5.6773   
20                          people_aot  0.015507  1.036418  Topic20  -5.6773   
21                       progress_make  0.015507  1.691663  Topic20  -5.6773   
25                  group_member_speak  0.015507  1.064983  Topic20  -5.6773   
26                        member_speak  0.015507  1.064983  Topic20  -5.6773   

     loglift  
9    30.0000  
73   29.0000  
10   28.0000  
74   27.0000  
201  26.0000  
..       ...  
19   -0.1916  
20   -0.1916  
21   -0.6816  
25   -0.2188  
26   -0.2188  

[959 rows x 6 columns], token_table=      Topic      Freq                           Term
term                                                
184       1  0.865198              activity_quantify
31        3  0.916121              activity_want_see
195       5  0.938982  advice_recommendation_improve
151       1  0.865198              align_value_bring
15        8  0.964862           amount_progress_make
...     ...       ...                            ...
72        2  0.643485                        work_do
208       3  0.916121                   work_do_need
167       3  0.916121                    work_ensure
49        2  0.879609                      work_give
200       5  0.938982          work_progress_do_work

[183 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 10, 19, 13, 9, 16, 15, 20, 14, 5, 3, 18, 2, 17, 11, 8, 6, 12, 1, 7])

# **Concordance**

In [39]:
all_text_57 = df.Q57.str.cat(sep = ' ') 
text = nltk.Text(nltk.word_tokenize(all_text_57))
text.concordance("progress")

Displaying 18 of 18 matches:
Progress. . Everyone 's task progress and plan , as well as current prob
unting for that in their designs . progress of the project is the collaboratio
 as each member 's report on their progress . In terms of computer science ass
comfortable speaking The amount of progress made at the end of each meeting , 
re the next meeting . Current work progress is an important expression of how 
ration is coming along , such as a progress chart . Productivity , constructiv
‚Äôre struggling with , as well as progress updates . See other people 's reac
it relates to the larger picture . Progress of a project and blockers that mem
eople are within the group and the progress that the group is making towards a
ollaborating . I would like to see progress . IdkIdkIdkIdkIdkIdkIdkIdkIdkIdkId
 burn-up chart is useful to see if progress is accelerating or slowing Nothing
ative and quantitative level . The progress of what they are working on , as w
elp . Are they happy about on

In [40]:
#------topic modeling for question 58 -----------
lda_model,corpus,id2word = run_topic_modeling(df.Q58.str.cat(sep = ' '))
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.252912 -0.108683       1        1  12.183808
10     0.207684  0.155232       2        1   8.175520
3     -0.238843  0.051919       3        1   7.097455
1     -0.247308 -0.066064       4        1   7.064719
5      0.103423 -0.210333       5        1   6.561594
13    -0.054192 -0.226447       6        1   6.442840
12    -0.146875 -0.155917       7        1   6.413558
4      0.201488  0.037608       8        1   5.529431
7      0.035684  0.220276       9        1   5.190567
0      0.012622 -0.148737      10        1   4.507932
14     0.109980 -0.072126      11        1   4.361659
17    -0.063556  0.177886      12        1   4.288521
8      0.084978  0.115247      13        1   4.144157
16    -0.144104  0.133245      14        1   4.141976
9      0.005848  0.093218      15        1   3.255555
2     -0.083728  0.039040      16        1   2.839661
18    -0.035771 -0.038716      17        1   2.317923
11    -0.032611  0.006882      18        1   2.286986
15     0.032377 -0.008975      19        1   2.193252
6     -0.000006  0.005444      20        1   1.002884, topic_info=                           Term      Freq     Total Category  logprob  loglift
21                      job_try  3.000000  3.000000  Default  30.0000  30.0000
11                   time_spend  3.000000  3.000000  Default  29.0000  29.0000
54     collaboration_would_like  1.000000  1.000000  Default  28.0000  28.0000
27             collaboration_go  2.000000  2.000000  Default  27.0000  27.0000
110               progress_make  2.000000  2.000000  Default  26.0000  26.0000
..                          ...       ...       ...      ...      ...      ...
25               finish_job_try  0.011343  1.705154  Topic20  -5.3279  -0.4105
26                 group_finish  0.011343  1.080359  Topic20  -5.3279   0.0459
27             collaboration_go  0.011343  2.821307  Topic20  -5.3279  -0.9140
28              individual_feel  0.011343  1.739966  Topic20  -5.3279  -0.4307
29   reflection_individual_feel  0.011343  1.186603  Topic20  -5.3279  -0.0479

[964 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
179       5  0.925618  ability_communicate
170      10  1.007703    activity_quantify
186       7  0.917523       advice_improve
70        9  0.981854       amount_work_do
123       4  0.907281  amount_work_do_post
...     ...       ...                  ...
89        1  0.842742       work_determine
71        9  0.981854              work_do
124       4  0.907281         work_do_post
93        3  0.922863   work_group_produce
194       7  0.917523      work_person_put

[186 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 11, 4, 2, 6, 14, 13, 5, 8, 1, 15, 18, 9, 17, 10, 3, 19, 12, 16, 7])

# **Concordance**

In [ ]:
all_text_58 = df.Q58.str.cat(sep = ' ') 
text = nltk.Text(nltk.word_tokenize(all_text_58))
text.concordance("progress")

In [43]:
#------topic modeling for question 59 -----------
lda_model,corpus,id2word = run_topic_modeling(df.Q59.str.cat(sep = ' '),'text')
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.103647  0.141903       1        1  13.719511
7     -0.126874 -0.097007       2        1   8.828735
18    -0.038035  0.227665       3        1   8.579373
16    -0.225301 -0.004389       4        1   7.170312
15    -0.171106  0.157808       5        1   7.001601
10     0.150445  0.216052       6        1   6.935869
2      0.310623 -0.028602       7        1   6.443850
13    -0.214778  0.067524       8        1   6.117106
0      0.014998  0.160493       9        1   5.051802
3     -0.189693 -0.161120      10        1   3.931342
1     -0.056015 -0.233466      11        1   3.487879
12     0.064346  0.044002      12        1   3.094657
6     -0.056584  0.026822      13        1   3.006815
8      0.099285 -0.202463      14        1   2.779378
19     0.142620 -0.105744      15        1   2.732419
14     0.168080  0.022712      16        1   2.610985
9      0.136975  0.104094      17        1   2.593446
11     0.018872 -0.172008      18        1   2.407700
4      0.008331 -0.057059      19        1   1.891670
5      0.067459 -0.107215      20        1   1.615549, topic_info=              Term       Freq      Total Category  logprob  loglift
15         improve  20.000000  20.000000  Default  30.0000  30.0000
255      would_use  27.000000  27.000000  Default  29.0000  29.0000
65   collaboration  35.000000  35.000000  Default  28.0000  28.0000
33            work  27.000000  27.000000  Default  27.0000  27.0000
14           group  34.000000  34.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
8           course   0.023441   1.442763  Topic20  -6.7523   0.0056
9        efficient   0.023441   3.848977  Topic20  -6.7523  -0.9756
10             far   0.023441   1.442690  Topic20  -6.7523   0.0057
11        feedback   0.023441   5.198449  Topic20  -6.7523  -1.2762
13            gain   0.023441   1.525799  Topic20  -6.7523  -0.0503

[913 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
345       2  0.450990          able
345       8  0.450990          able
386       8  0.707459        access
166       4  0.692775    accomplish
454       5  0.361259  accomplished
...     ...       ...           ...
255      16  0.036572     would_use
255      17  0.036572     would_use
165       1  0.341652         wrong
165       2  0.341652         wrong
165      14  0.341652         wrong

[858 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 8, 19, 17, 16, 11, 3, 14, 1, 4, 2, 13, 7, 9, 20, 15, 10, 12, 5, 6])

# **Concordance**

In [44]:
all_text_59 = df.Q59.str.cat(sep = ' ') 
text = nltk.Text(nltk.word_tokenize(all_text_59))
text.concordance("feedback")

Displaying 6 of 6 matches:
orking in a small group Using this feedback from collaborators in retrospect t
estigate any instances of negative feedback to see if the group member was har
 if we did by the next meeting ) . feedback on group dynamics can help improve
ld use this information to provide feedback to the group as a whole and each i
or further collaboration . Provide feedback for the group members on how they 
the collaboration . implement this feedback to future collaboration efforts I 


In [ ]:
# ----- TOPIC MODELING OF ALL RESPONSES----

In [ ]:
#------topic modeling for question 59 -----------
all_responses = df.Q51.str.cat(sep = ' ')+ df.Q52.str.cat(sep = ' ')+df.Q57.str.cat(sep = ' ')+df.Q55.str.cat(sep = ' ')+df.Q58.str.cat(sep = ' ')+df.Q59.str.cat(sep = ' ')
lda_model,corpus,id2word = run_topic_modeling(all_responses)
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')#,mds='mmds'
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.231591  0.176727       1        1  6.943834
19     0.086504 -0.249353       2        1  6.752833
15    -0.136933 -0.138665       3        1  6.060958
4     -0.066376  0.232980       4        1  5.806320
11     0.161855 -0.160093       5        1  5.772400
14     0.220282 -0.068791       6        1  5.373481
8      0.023826  0.188981       7        1  5.276458
1     -0.131151 -0.194635       8        1  5.157917
12    -0.231824  0.080005       9        1  5.064256
9      0.111733  0.179070      10        1  5.020335
3      0.204030  0.034612      11        1  4.875035
7     -0.002624  0.074034      12        1  4.670450
6     -0.153165 -0.015984      13        1  4.589315
17    -0.138463  0.161288      14        1  4.529904
2     -0.013196 -0.197750      15        1  4.436091
13    -0.109190  0.077851      16        1  4.281975
16     0.012057 -0.101336      17        1  4.185132
5     -0.200595 -0.059487      18        1  4.007997
10     0.031711 -0.024771      19        1  3.729413
18     0.099931  0.005318      20        1  3.465894, topic_info=                    Term       Freq      Total Category  logprob  loglift
13               job_try  16.000000  16.000000  Default  30.0000  30.0000
14   participate_job_try  10.000000  10.000000  Default  29.0000  29.0000
6             time_spend  19.000000  19.000000  Default  28.0000  28.0000
15        finish_job_try   6.000000   6.000000  Default  27.0000  27.0000
37            effort_put   4.000000   4.000000  Default  26.0000  26.0000
..                   ...        ...        ...      ...      ...      ...
340          people_talk   0.987443   3.857141  Topic20  -4.0758   1.9996
467     collaboration_go   0.507956   3.211934  Topic20  -4.7405   1.5180
510        group_produce   0.507956   1.566048  Topic20  -4.7405   2.2363
554             thing_go   0.507956   3.258514  Topic20  -4.7405   1.5036
785       work_would_use   0.507956   1.618472  Topic20  -4.7405   2.2034

[787 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
233       4  0.458870  ability_communicate
233      16  0.458870  ability_communicate
298       2  0.849813          ability_get
333      16  0.932386        act_idea_find
39        9  0.898221        activity_take
...     ...       ...                  ...
744      12  0.904086     work_try_improve
112      16  0.932386      work_would_make
785       5  0.617867       work_would_use
785      20  0.617867       work_would_use
787       9  0.898221      workload_ensure

[790 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 20, 16, 5, 12, 15, 9, 2, 13, 10, 4, 8, 7, 18, 3, 14, 17, 6, 11, 19])

In [21]:
sia = SentimentIntensityAnalyzer()
positive_review_ids = nltk.corpus.movie_reviews.fileids(categories=["pos"])
negative_review_ids = nltk.corpus.movie_reviews.fileids(categories=["neg"])
all_review_ids = positive_review_ids + negative_review_ids

In [22]:
def is_positive(review_id: str) -> bool:
    """True if the average of all sentence compound scores is positive."""
    text = nltk.corpus.movie_reviews.raw(review_id)
    #print(text)
    scores = [
        sia.polarity_scores(sentence)["compound"]
        for sentence in nltk.sent_tokenize(text)
    ]
    return mean(scores) > 0


In [23]:
correct = 0
for review_id in all_review_ids:
     if is_positive(review_id):
        if review_id in positive_review_ids:
             correct += 1
     else:
         if review_id in negative_review_ids:
             correct += 1

print(F"{correct / len(all_review_ids):.2%} correct")

64.05% correct


In [24]:
unwanted = nltk.corpus.stopwords.words("english")

unwanted.extend([w.lower() for w in nltk.corpus.names.words()])


def skip_unwanted(pos_tuple):

    word, tag = pos_tuple

    if not word.isalpha() or word in unwanted:

        return False

    if tag.startswith("NN"):

        return False

    return True


positive_words = [word for word, tag in filter(

    skip_unwanted,

    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["pos"]))

)]

negative_words = [word for word, tag in filter(

    skip_unwanted,

    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["neg"]))

)]


In [25]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}


In [ ]:
# USING MOVIE REVIEW SAMPLES TO AS TRAINIG DATA

In [26]:
def extract_features(text):
    features = dict()
    wordcount = 0
    compound_scores = list()
    positive_scores = list()

    for sentence in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sentence):
            if word.lower() in positive_fd:
                wordcount += 1
        compound_scores.append(sia.polarity_scores(sentence)["compound"])
        positive_scores.append(sia.polarity_scores(sentence)["pos"])

    # Adding 1 to the final compound score to always have positive numbers
    # since some classifiers  don't work with negative numbers.
    features["mean_compound"] = mean(compound_scores) + 1
    features["mean_positive"] = mean(positive_scores)
    features["wordcount"] = wordcount

    return features


In [27]:
positives = 0
negatives = 0
for resp in df.Q51:
  if resp is not nan:
    features = extract_features(resp)
    if features["mean_positive"] > 0:
      positives+=1
    else:
       negatives+=1 
    #print(features)
print("Sentiments --- Positives: ", positives, " Negatives: ",negatives)    


Sentiments --- Positives:  113  Negatives:  91


In [30]:
all_text = df.Q51.str.cat(sep = ' ') 
text = nltk.Text(nltk.word_tokenize(all_text))
text.concordance("peer", lines=5)

Displaying 5 of 12 matches:
 been present at the team meetings . peer reviews Are they actually participat
tributed to each part of a project . Peer reviews as well . How much time they
esent on the parts they made maybe ? Peer evals potentially Time spent contrib
ossbile but is not always accurate . Peer reviews ( with the understanding tha
esult of their part of work leads to Peer reviews of performance would be usef


In [ ]:
concordance_list = text.concordance_list("group", lines=5)
for entry in concordance_list:
    print(entry.line)

In [ ]:
token = nltk.word_tokenize(all_text_58)
stopwords = nltk.corpus.stopwords.words("english")
words_58 = [w for w in token if w.lower() not in stopwords and w.isalpha()]
text_1 = nltk.Text(words_58)
fd = text_1.vocab()
fd.tabulate(20)

collaboration         would         group           see          like          work       members          well       project   information        people          felt        member          much       Nothing      everyone        person         think          time       quality 
           58            56            51            46            45            29            28            22            19            18            16            16            15            14            14            13            13            12            11            11 


In [ ]:
finder = nltk.collocations.BigramCollocationFinder.from_words(words_58)
finder.ngram_fd.most_common(10)

In [ ]:
nltk.pos_tag(['person'],tagset='universal')

In [ ]:
w = nlp('person')

for token in w:
  print(token.lemma_)